In [1]:
import pandas as pd
import numpy as np
import sys

import gzip, pickle

In [2]:
sys.path.append("./../../../../routes_laboratory/")

In [3]:
from models.rich_vrp.agent_type import AgentType
from models.rich_vrp.agent import Agent
from models.rich_vrp.job import Job
from models.point import Point
from dataclasses import dataclass

# Агенты

In [4]:
curier_xl = pd.read_excel('./Курьеры_13.xlsx')

In [5]:
curier_xl.head(2)

,Интервал работы,Сотрудник,Должность,Стоимость 1 заказа,Приоритет
0,16-22,Акперов Ильгар Натиг Оглы,Курьер,5,NaN
1,9-21,Бердников Евгений Леонидович,Водитель,10,NaN


In [6]:
@dataclass
class EAptekaAgent(AgentType):
    id : str
    name : str
    job_name : str
    priority : int

In [7]:
def to_string_time(t):
    if t == 'круглосуточно':
        return '00:00', '24:00'
    else:
        s, f = map(int, t.split('-'))
        return str(s)+':00', str(f)+':00'

In [8]:
agents = list([])
for i, row in curier_xl.iterrows():
    tAgent = EAptekaAgent(id = i,
                          name = row['Сотрудник'], 
                          job_name = row['Должность'], 
                          priority = row['Приоритет'])
    
    costs = {'fixed' : row['Стоимость 1 заказа']}
    work_start, work_finish = to_string_time(row['Интервал работы'])
    agents.append(Agent(type = tAgent,
                         costs=costs,
                         value = None,
                         start_time=work_start,
                         end_time=work_finish,
                         start_place=None,
                         end_place=None
    ))

In [9]:
with gzip.open('agents.pkl.gz', 'wb') as f:
        pickle.dump(agents, file=f, protocol=pickle.HIGHEST_PROTOCOL)

# Склады

In [10]:
depots_xl = pd.read_excel('./Склады.xlsx')

In [11]:
depots_xl.head(2)

,Наименование,Адрес,График работы,Широта,Долгота
0,"Мск, 1905 года 7","Москва г, 1905 года ул, дом № 7, строение 1 ...",09-21,55.762100,37.558670
1,"Мск, Бауманская 62-66","Москва г, Бауманская ул, дом № 62-66, строение...",09-21,55.766116,37.678837


In [12]:
@dataclass
class EAptekaDepot:
    id : int
    name : str
    addres : str
    longitude : float
    latitude : float
    start_time : str
    end_time : str

In [13]:
depots = list([])
coords = list([])

depots_name_id_dict = dict({})
for i, row in depots_xl.iterrows():
    start_time, end_time = to_string_time(row['График работы'])
    depots_name_id_dict[row['Наименование']] = i
    
    coords.append(Point(lon = float(row['Долгота']), lat = float(row['Широта'])))
    
    depots.append(EAptekaDepot(id =i,
                                name = row['Наименование'],
                                addres = row['Адрес'],
                                longitude=float(row['Долгота']),
                                latitude=float(row['Широта']),
                                start_time = start_time,
                                end_time = end_time
    ))

In [14]:
with gzip.open('depots.pkl.gz', 'wb') as f:
        pickle.dump(depots, file=f, protocol=pickle.HIGHEST_PROTOCOL)

# Заказы

In [15]:
orders_lx = pd.read_excel('./Заказы_13.xlsx')
orders_coords_xl = pd.read_excel('./eapteka_result.xlsx')

In [16]:
orders_lx.head(2)

,Номер,ДатаДоставки,Склад,ЗонаДоставки,ИнтервалДоставки,АдресДоставки,СуммаДокумента,ВесЗаказа,ОбъемЗаказа,КомментарийКлиента,Приоритет,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,10848260,13.09.2020 0:00:00,"Мск, Марьина роща склад",Я1,Доставка с 12:00 по 14:00,"Москва, Королев г, Коминтерна улица, д.15",4900.0,NaN,0.140,"ПЛАНЕТА ЗДОРОВЬЯ Королев, Коминтерна ул, 15, Ч...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9133952,13.09.2020 0:00:00,"Мск, Марьина роща склад",55,Доставка с 09:00 по 18:00,"Москва, Ангарская улица, 67к1",1486.0,0.972,8.084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
orders_coords_xl.head(2)

,Unnamed: 0,Адрес из Excel,Коммент из Excel,Адрес из геокодера,Долгота,Широта
0,0,"Москва, Королев г, Коминтерна улица, д.15","ПЛАНЕТА ЗДОРОВЬЯ Королев, Коминтерна ул, 15, Ч...","Россия, Московская область, Королёв, улица Ком...",55.923992,37.813854
1,1,"Москва, Ангарская улица, 67к1",NaN,"Россия, Москва, Ангарская улица, 67 к1",55.885930,37.525600


In [18]:
orders_all_xl = pd.concat([orders_lx, orders_coords_xl.reindex(orders_lx.index)], axis=1)

In [19]:
#Проверка.
for row in orders_all_xl[['Адрес из Excel', 'АдресДоставки']].values:
    if not (row[0] == row[1]):
        print('Не совпадают!:', row[0], row[1])

In [20]:
from datetime import datetime, timedelta

In [21]:
@dataclass
class EAptekaOrder(Job):
    order_id : int
    #id : int
    geocoder_addres : str
    longitude : float
    latitude : float
    #tw_start : datetime
    #tw_end : datetime
    order_zone: str
    depot_id : int
    weight : float
    volume : float
    price : float
    comment : str

In [22]:
time_dict = dict({})
l = orders_all_xl['ИнтервалДоставки'].values

for st in set(l):
    tup = ['00:00', '00:00']
    
    if st.split()[2] not in {'с', 'по'}:
        tup[0] = st.split()[2]
        
    if st.split()[-1] not in {'с', 'по'}:
        tup[1] = st.split()[-1]

    time_dict[st] = tup

In [23]:
#Проверка:
time_dict

{'Доставка с 21:00 по 23:35': ['21:00', '23:35'],
 'Доставка с 09:00 по 18:00': ['09:00', '18:00'],
 'Доставка с 10:00 по 11:00': ['10:00', '11:00'],
 'Доставка с 11:18 по 17:18': ['11:18', '17:18'],
 'Доставка с 09:00 по 13:00': ['09:00', '13:00'],
 'Доставка с 11:00 по 15:00': ['11:00', '15:00'],
 'Доставка с 17:00 по 23:00': ['17:00', '23:00'],
 'Доставка с 12:00 по 16:00': ['12:00', '16:00'],
 'Доставка с 09:00 по 15:00': ['09:00', '15:00'],
 'Доставка с 20:00 по ': ['20:00', '00:00'],
 'Доставка с 18:00 по 23:00': ['18:00', '23:00'],
 'Доставка с 15:00 по 21:00': ['15:00', '21:00'],
 'Доставка с 06:00 по 09:00': ['06:00', '09:00'],
 'Доставка с 13:00 по 19:00': ['13:00', '19:00'],
 'Доставка с 21:00 по ': ['21:00', '00:00'],
 'Доставка с 09:00 по 12:00': ['09:00', '12:00'],
 'Доставка с 10:00 по 16:00': ['10:00', '16:00'],
 'Доставка с 08:00 по 14:00': ['08:00', '14:00'],
 'Доставка с 14:00 по 20:00': ['14:00', '20:00'],
 'Доставка с 12:00 по 18:00': ['12:00', '18:00'],
 'Доставка

In [24]:
def make_tw(date, tw):
    l, r = time_dict[tw]
    l = datetime.strptime(date[:-len(l)]+l, '%d.%m.%Y %H:%M:%S')
    r = datetime.strptime(date[:-len(r)]+r, '%d.%m.%Y %H:%M:%S')
    
    if time_dict[tw][1] < time_dict[tw][0]:
        r = r + timedelta(days=1)
        
    return l, r

In [25]:
orders = list([])
start_id = len(coords)

for i, row in orders_all_xl.iterrows():
        tw_start, tw_end = make_tw(row['ДатаДоставки'], row['ИнтервалДоставки'])
        try:
            coords.append(Point(lat = float(row['Широта']), lon = float(row['Долгота'])))
        
            orders.append(EAptekaOrder(
                order_id  = row['Номер'],
                id= i+start_id,
                geocoder_addres = row['Адрес из геокодера'],
                longitude = float(row['Долгота']),
                latitude = float(row['Широта']),
                tw_start =tw_start,
                tw_end = tw_end,
                order_zone = row['ЗонаДоставки'],
                depot_id = depots_name_id_dict[row['Склад']],
                weight  = float(row['ВесЗаказа']),
                volume = float(row['ОбъемЗаказа']),
                price = float(row['СуммаДокумента']),
                comment = row['КомментарийКлиента'],
                delay = None,
                demand = None,

            ))
        except:
            print(i, row['Долгота'], row['Широта'])

In [26]:
with gzip.open('orders.pkl.gz', 'wb') as f:
        pickle.dump(orders, file=f, protocol=pickle.HIGHEST_PROTOCOL)

with gzip.open('orders_coords.pkl.gz', 'wb') as f:
        pickle.dump(coords, file=f, protocol=pickle.HIGHEST_PROTOCOL)

# Матрица расстояний

In [36]:
for i, p in enumerate(coords):
    if np.isnan(p.lat) or np.isnan(p.lon):
        print(i, "Nans detected", p.lat, p.lon)
    elif not (54.288066 < p.lat < 57.172201) and (34.619289 < p.lon < 39.724750):
        print('Номер в Excel:', i-11, '/ Strange point detected', p.lat, p.lon)

Номер в Excel: 466 / Strange point detected 59.106968 38.778847
Номер в Excel: 500 / Strange point detected 51.666855 36.089752000000004
Номер в Excel: 573 / Strange point detected 50.642296 36.566615999999996
Номер в Excel: 598 / Strange point detected 57.3357 38.2412
Номер в Excel: 753 / Strange point detected 45.351875 37.103535
Номер в Excel: 755 / Strange point detected 60.416340000000005 39.43523
Номер в Excel: 799 / Strange point detected 53.770416000000004 38.11027
Номер в Excel: 880 / Strange point detected 51.725533 36.099922
Номер в Excel: 1012 / Strange point detected 45.351875 37.103535
Номер в Excel: 1242 / Strange point detected 50.61574 38.68185
Номер в Excel: 1301 / Strange point detected 51.778397 36.20394
Номер в Excel: 1356 / Strange point detected 58.033843999999995 38.846270000000004
Номер в Excel: 1426 / Strange point detected 51.267193 37.5214
Номер в Excel: 1431 / Strange point detected 51.234035 35.669556
Номер в Excel: 1691 / Strange point detected 52.6752550

In [28]:
from geo.martices.osrm import get_osrm_matrix
from geo.martices.osrm import fix_matrix
import os

Первые индексы матрицы отвечают складам, а дельше уже заказам.

In [29]:
os.environ["OSRM_HOST"] = "desktop"
os.environ["OSRM_PORT"] = '5000'

In [30]:
route_matrix = get_osrm_matrix(tuple(coords))

Не нашли в кеше. Обновляем матрицу расстояний...


In [33]:
tuple_coords = list([])
for p in coords:
    tuple_coords.append((p.lat, p.lon))

In [34]:
route_matrix = fix_matrix(route_matrix, tuple_coords, 1.23)

In [35]:
with gzip.open('route_matrix.pkl.gz', 'wb') as f:
        pickle.dump(route_matrix, file=f, protocol=pickle.HIGHEST_PROTOCOL)